In [ ]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime as dt
import smartsheet
import os
import re
from sql_utliz import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [ ]:
api_key = 'smartsheet_key'
monthly_billing = 'smartsheet_id'
source_billable_records_sheet = 'smartsheet_id'
variance_report = 'smartsheet_id'

# Load the Jira tables

In [ ]:
# Database connection settings
server = 'server_path'
database = 'database_name'
username = 'username'
password = 'password'
BIC_Issues = 'dbo.table_name'
Top_Level_Issues = 'tablename' # load the BIC_Top_Level_Issues table to extact the required data
BIC_User_Time ='tablename'

# Establishing the database connection
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'
conn = pyodbc.connect(conn_str)

# SQL query to fetch the view data
query1 = f'SELECT * FROM {BIC_Issues}'
query2 = f'SELECT * FROM {Top_Level_Issues}'
query3 = f'SELECT * FROM {BIC_User_Time}'

# Executing the query and fetching the data
bic_issues_df = pd.read_sql_query(query1, conn)
top_level_issues_df = pd.read_sql_query(query2, conn)
bic_user_time_df = pd.read_sql_query(query3, conn)

# Closing the database connection
conn.close()


# Identify top level records that have an index

In [ ]:
target_project_key_types = ['UTT', 'RDS']
target_project_key_records = top_level_issues_df[top_level_issues_df.Project_Key.isin(target_project_key_types)]
top_level_records_with_index = target_project_key_records[target_project_key_records.Research_Info__Account_Num.notna()]
top_level_records_with_index_id_list = list(set(list(top_level_records_with_index.Issue_ID)))
len(top_level_records_with_index_id_list)

In [ ]:

# slice out of the top level table the fields needed
top_level_required_fields = top_level_records_with_index[['Issue_ID', 
                                                          'Issue_Ticket_Number',
                                                          'Project_Name',
                                                          'Project_Key',
                                                          'Issue_Status',
                                                          'Research_Info__Project_Title',
                                                          'Research_Info__PI_Name',
                                                          'Research_Info__Project_Description',
                                                          'Research_Info__IRB_Num',
                                                          'Research_Info__Account_Num']]


# slice out only the required fields from the bic_issues table and filter down to the target records
bic_issues_required_fields = bic_issues_df[['Issue_ID',
                                            'Parent_Issue_ID',
                                            'Issue_Ticket_Number',
                                            #'Billable_Time_Seconds',
                                            'Project_Key',
                                            'Issue_Summary',
                                            'Issue_Description',
                                            'Issue_Status']]
                                            #'Created_Date',
                                            #'Updated_Date',
                                            #'Due_Date',
                                            #'Resolution_Date']]


# update the parent table field titles so they can be merged with without causing title problems
top_level_required_fields.rename(columns={'Issue_Ticket_Number':'Issue_Ticket_Number_parent', 
                                          'Issue_Status':'Issue_Status_parent',
                                          'Project_Key':'Project_Key_parent',                                           
                                          'Issue_Summary':'Issue_Summary_parent',
                                          'Issue_Description':'Issue_Description_parent',
                                          'Issue_Status':'Issue_Status_parent',
                                          'Created_Date':'Created_Date_parent',
                                          'Updated_Date':'Updated_Date_parent',
                                          'Due_Date':'Due_Date_parent',
                                          'Resolution_Date':'Resolution_Date_parent'}, inplace=True)



# Remove subtask records that are not billed to the client

In [ ]:
remove_title_list = ['SLA (Sub of RDS','SLA','Exploratory (Sub of RDS','Quality Control (Sub of RDS','Compliance Review (Sub of RDS']

def records_to_remove(text):
    for keyword in remove_title_list:
        if keyword in text:
            return 1
    return 0

# add a column that identifies if the 'Issue_Summary' field contains the text SLA and returns a 1 if SLA is in the text
bic_issues_required_fields['remove_record'] = bic_issues_required_fields.apply(lambda x: records_to_remove(x.Issue_Summary), axis=1)

# filter to remove records that should not have time aggrogated for billing 
bic_issues_id_remove_df = bic_issues_required_fields[bic_issues_required_fields.remove_record == 1]
bic_issues_id_remove_list = list(set(list(bic_issues_id_remove_df.Issue_ID))) 
bic_issues_required_fields = bic_issues_required_fields[bic_issues_required_fields.remove_record == 0]

#bic_issues_required_fields.drop(columns=['remove_record'], inplace=True)

In [ ]:


# replace the "NaN" values in the Parent_Issue_ID field with the values from the Issue_ID field to help with identifiing and merging
# since it appears the missing values are due to these records being the parent records
bic_issues_required_fields['Parent_Issue_ID'].fillna(bic_issues_required_fields['Issue_ID'], inplace=True)

bic_issues_target_records = bic_issues_required_fields[bic_issues_required_fields.Parent_Issue_ID.isin(top_level_records_with_index_id_list)]
len(bic_issues_target_records)
#bic_issues_target_records

In [ ]:
# slice out the required fields from the bic_user_time df
bic_user_time_required_fields = bic_user_time_df[['WorkLog_ID', 
                                                  'Top_Level_Issue_ID', 
                                                  'Work_Logged_Issue_ID', 
                                                  'User_Display_Name', 
                                                  'Work_Start',
                                                  'Time_Worked_Seconds', 
                                                  'Time_Is_Billable']]

In [ ]:
# because the bic_user_time table has a column 'Top_Level_Issue_ID' that list the parent record for the logged work which appears never to be blank
# and the "Work_Logged_Issue_ID appears to log where the work was attributed to, which could be the parent ticket, 
# return bic user time records that have parent IDs that include a billable index and have time listed as 'billable'
bic_user_time_with_billable_index = bic_user_time_required_fields[(bic_user_time_required_fields.Top_Level_Issue_ID.isin(top_level_records_with_index_id_list)) & (bic_user_time_required_fields.Time_Is_Billable==1)]

# we then need to remove work time attributed to subtasks that are not billable using the listing identified prior
bic_user_time_with_billable_index = bic_user_time_with_billable_index[~bic_user_time_with_billable_index.Work_Logged_Issue_ID.isin(bic_issues_id_remove_list)]
len(bic_user_time_with_billable_index)

In [ ]:
# add a column that derivers the billable time in hours and fractions of hours
bic_user_time_with_billable_index['Time_Worked_Hours'] = (bic_user_time_with_billable_index.Time_Worked_Seconds/60)/60

# convert the 'Work_Start' field to datetime to enable sorting by date
bic_user_time_with_billable_index['Work_Start'] = pd.to_datetime(bic_user_time_with_billable_index['Work_Start'])

bic_user_time_with_billable_index.head(3)

# Add a field that applies the 8 hours gratis based on the first 8 billable hours

In [ ]:
# get a unique listing of the Top_Level_Issue_ID so this can be used as a filter to return only records for the parent ticket
parent_ticket_unique_list = list(set(list(bic_user_time_with_billable_index.Top_Level_Issue_ID)))


# helper funtion for accounting for the gratis time
def distribute_gratis_hours(dataframe, gratis):
    # Check if the DataFrame is empty or gratis is non-positive
    if dataframe.empty or gratis <= 0:
        return dataframe

    # Initialize a new column "gratis_hour" with zeros
    dataframe["gratis_time"] = 0

    # Iterate through rows and distribute gratis hours
    for index, row in dataframe.iterrows():
        time_worked_hours = row["Time_Worked_Hours"]
        remaining_gratis = gratis - row["gratis_time"]

        # If there are gratis hours remaining and enough "Time_Worked_Hours" in this row
        if remaining_gratis > 0 and time_worked_hours > 0:
            # Calculate the amount of gratis hours to add to this row
            if time_worked_hours >= remaining_gratis:
                dataframe.at[index, "gratis_time"] += remaining_gratis
            else:
                dataframe.at[index, "gratis_time"] += time_worked_hours

            # Update the remaining gratis hours
            gratis -= dataframe.at[index, "gratis_time"]

    return dataframe


# loop over the billable records that are filtered per the parent ticket and apply the gratis time
df_hold = []
for parent_ticket in parent_ticket_unique_list:
    
    # return a listing of records related by the parent ticket 
    parent_record_set = bic_user_time_with_billable_index[bic_user_time_with_billable_index.Top_Level_Issue_ID == parent_ticket]
    parent_record_set = parent_record_set.sort_values(by=['Work_Start'], ascending=True)
    
    # get the len of the parent record set so this can be used for the while loop
    parent_record_set_len = (parent_record_set)
    
    parent_record_set = distribute_gratis_hours(parent_record_set, 8)        
    
    df_hold.append(parent_record_set)
    

# get an updated data frame that adds the column gratis_time that applies the required gratis time per record
bic_user_time_with_billable_index = pd.concat(df_hold)
bic_user_time_with_billable_index.head(3)

In [ ]:
# create a helper column that identifies if Top_Level_Issue_ID and Work_Logged_Issue_ID have the same value as this indicates the record 
# has time applied to the parent ticket. The records with matching values will be sliced from those without so the parent ticket fields can 
# be merged with the parent ticket data then the records with values that do not match can have the parent ticket data merged based on the 
# "Top_Level_Issue_ID field and then have the subtask data merged on the "Work_Logged_Issue_ID" field. Finally, the two seperate DFs will be 
# concat resulting in records that are subtasks having both parent and subtask features while records with only parent ticket data will have 
# missing values for any subtask fields

bic_user_time_with_billable_index['matching_issue_ids'] = bic_user_time_with_billable_index.apply(lambda x: 1 if x.Top_Level_Issue_ID == x.Work_Logged_Issue_ID else 0, axis=1)

issue_ids_match_records = bic_user_time_with_billable_index[bic_user_time_with_billable_index.matching_issue_ids == 1]
issue_ids_match_records.drop(columns=['matching_issue_ids'], inplace=True)

issue_ids_no_match_records = bic_user_time_with_billable_index[bic_user_time_with_billable_index.matching_issue_ids == 0]
issue_ids_no_match_records.drop(columns=['matching_issue_ids'], inplace=True)

merge_issue_ids_match_records_parent_features = pd.merge(issue_ids_match_records,top_level_required_fields, left_on='Work_Logged_Issue_ID', right_on='Issue_ID', how='left')
merge_issue_ids_match_records_sub_feature = pd.merge(issue_ids_no_match_records,bic_issues_required_fields, left_on='Work_Logged_Issue_ID', right_on='Issue_ID', how='left')
merge_issue_ids_match_records_sub_feature = pd.merge(merge_issue_ids_match_records_sub_feature,top_level_required_fields, left_on='Top_Level_Issue_ID', right_on='Issue_ID', how='left')

final_record_set = pd.concat([merge_issue_ids_match_records_parent_features,merge_issue_ids_match_records_sub_feature], sort=False)

# Create a new column 'month_year_of_hours_worked' with year and month
final_record_set['Work_Start'] = pd.to_datetime(final_record_set['Work_Start'])
final_record_set['month_year_of_hours_worked'] = final_record_set['Work_Start'].dt.to_period('M')
# create a filter to only use records from 2 months ago (current month +1) to account for changes with the billing
target_processing_month_year = pd.to_datetime(dt.now()).to_period('M')-2

# Filter records with the same year and month as the current year and month
final_record_set = final_record_set[final_record_set['month_year_of_hours_worked'] <= target_processing_month_year]
final_record_set = final_record_set.sort_values(by='Work_Start', ascending=False)


In [ ]:


def save_dataframe_as_excel(df, index, month_effort_accrued):
    # Check if the 'delete_me' directory exists, and create it if not
    if not os.path.exists('delete_me'):
        os.makedirs('delete_me')

    # Define a regular expression pattern for a valid index
    valid_index_pattern = r'^[A-Za-z]*\d+$'

    # Check if the index matches the valid pattern
    if re.match(valid_index_pattern, index):
        # Define the Excel file name based on the index and month_effort_accrued
        excel_file_name = f'delete_me/{index}_{month_effort_accrued}.xlsx'
    else:
        # If the index doesn't match the pattern, set it to "ERROR"
        index = "ERROR"
        # Define a generic Excel file name
        excel_file_name = f'delete_me/{index}_{month_effort_accrued}.xlsx'

    # Save the DataFrame as an Excel file
    df.to_excel(excel_file_name, index=False)

    return excel_file_name

In [ ]:

def calculate_hours_worked_per_month(df):
    # Convert 'Work_Start' column to datetime if it's not already
    df['Work_Start'] = pd.to_datetime(df['Work_Start'])
    
    # create a new field called hours_less_gratis that will be used to calculate the billable hours
    df['hours_less_gratis'] = df.Time_Worked_Hours - df.gratis_time
    
    # get a unique list of the values in Research_Info__Account_Num so they can be used as filter
    index_list = list(set(list(df.Research_Info__Account_Num)))
    
    # list to hold the output dfs 
    processed_dfs_list = []
    
    # use the index as a filter to return only records for that index
    for idx in index_list:
        
        index_df = df[df.Research_Info__Account_Num == idx]
        
        # get a list of the month_year_of_hours_worked dates to use as a filter to group the records
        work_date_list = list(set(list(index_df.month_year_of_hours_worked)))
        
        # list to hold the processed dfs
        loop_df_list = []
        
        
        # filter based on each of the available dates in the work_date_list
        for date in work_date_list:
            # filter records to group based on the month/year 
            date_df = index_df[index_df.month_year_of_hours_worked == date]
            
            monthly_sum = date_df.hours_less_gratis.sum()
            
            
            # Select the columns to slice/keep 
            columns_to_keep = ['Research_Info__Account_Num', 'Project_Key_parent', 'Issue_Ticket_Number_parent',
                               'Issue_Status_parent', 'Research_Info__Project_Title', 'Research_Info__PI_Name',
                               'Research_Info__Project_Description', 'Research_Info__IRB_Num', 'month_year_of_hours_worked']

            loop_result = date_df[columns_to_keep].iloc[:1]
            loop_result['hours_worked_in_month'] = monthly_sum
            loop_df_list.append(loop_result)
            
            # save the result as a pdf for invoicing 
            #save_dataframe_as_excel(loop_result, idx, date)
            save_dataframe_as_excel(date_df, idx, date)
            
        # concat the list
        concat_list = pd.concat(loop_df_list)
        processed_dfs_list.append(concat_list)
        
    results_df = pd.concat(processed_dfs_list)
    return results_df
        
                
        

In [ ]:


# Call the function to calculate hours worked per month
result_df = calculate_hours_worked_per_month(final_record_set)

In [ ]:


# add the date/time the records were processed 
batched_time = dt.now().strftime('%Y-%m-%d_%H:%M:%S')
result_df['processed_on'] = batched_time
final_record_set['processed_on'] = batched_time # records that were the original source records that were cleaned for processing
result_df = result_df.sort_values('month_year_of_hours_worked', ascending = True)

# add the field for RamsForce processing status
result_df['ramsforce_status'] = 'Ready to Process'

result_df.tail()

In [ ]:
def get_column_name_to_id(sheet_id, api_key):
    # Initialize the Smartsheet client
    smartsheet_client = smartsheet.Smartsheet(api_key)

    try:
        # Load the specified sheet
        sheet = smartsheet_client.Sheets.get_sheet(sheet_id)

        # Initialize an empty dictionary to store column name to column ID mapping
        column_name_to_id = {}

        # Iterate through columns in the sheet
        for column in sheet.columns:
            # Store the column name (title) as the key and column ID as the value
            column_name_to_id[column.title] = column.id

        return column_name_to_id

    except Exception as e:
        print(f"Error: {e}")
        return None
    


In [ ]:


def extract_and_load_to_dataframe_with_row_id(api_token, sheet_id):
    # Initialize the Smartsheet client
    ss_client = smartsheet.Smartsheet(api_token)

    try:
        # Get the sheet information to access column details
        sheet = ss_client.Sheets.get_sheet(sheet_id)
        columns = sheet.columns

        # Extract column headers from the sheet
        column_headers = [column.title for column in columns]

        # Load the entire sheet
        sheet = ss_client.Sheets.get_sheet(sheet_id, page_size=10000)
        rows = sheet.rows

        # Create a list to hold row data
        data = []

        # Extract data from each row and add to the list
        for row in rows:
            row_data = []
            
            # Add the Smartsheet row ID as the first column
            row_data.append(row.id)
            
            for cell in row.cells:
                row_data.append(cell.display_value)
            data.append(row_data)

        # Add "row_id" as the column name for the Smartsheet row ID
        column_headers.insert(0, "row_id")

        # Convert the list to a DataFrame with column headers
        df = pd.DataFrame(data, columns=column_headers)

        return df

    except Exception as e:
        print(f"Error: {e}")
        return None


In [ ]:
def add_records_to_smartsheet(df, api_key, sheet_unique_id):
    
    # get the smartsheet column ids 
    col_ids = get_column_name_to_id(sheet_unique_id, api_key)
    
    # create a copy of the input df
    scoped_records = df.copy()
    
    # Initialize the Smartsheet client
    smartsheet_client = smartsheet.Smartsheet(api_key)

    # Convert all columns to string data type
    new_records = scoped_records.astype(str)

    new_records_columns = list(set(list(scoped_records)))
    df_len = len(new_records)
    counter = 0
    # use a counter to iterate over the rows within the dataframe to load the data from each row into the smartsheet format
    while counter < df_len:
        # Specify cell values for one row
        row_a = smartsheet.models.Row()
        row_a.to_top = True
        # iterate over the field names in the scoped dataset and use these field names to call the 
        # key from the result dictionary that contains the smartsheet field ID associated with the key/title
        # and use this field ID to build the record data neede for smartsheet to post the row
        for column_name in new_records_columns:
            row_a.cells.append({
                'column_id': int(col_ids[column_name]), 
                'value': new_records.iloc[counter][column_name]
            })
        # Add rows to sheet
        response = smartsheet_client.Sheets.add_rows(
            sheet_id=sheet_unique_id,  
            list_of_rows=[row_a],
        )

        # Check for success or handle errors as needed
        if response.message == 'SUCCESS':
            print('Rows added successfully')
            counter = counter + 1
        else:
            print(f'Error: {response.message}')
            counter = counter + 1
    

In [ ]:


def add_excel_file_to_row(api_token, sheet_id, row_id, excel_file_path):
    try:
        # Initialize the Smartsheet client
        smartsheet_client = smartsheet.Smartsheet(api_token)

        # Check if the Excel file exists at the specified path
        if not os.path.isfile(excel_file_path):
            print(f"Excel file not found at path: {excel_file_path}")
            return False

        # Extract the file name from the path
        file_name = os.path.basename(excel_file_path)

        # Attach the Excel file to the row using smartsheet_client.Attachments.attach_file_to_row
        response = smartsheet_client.Attachments.attach_file_to_row(
            sheet_id,
            row_id,
            (file_name, open(excel_file_path, 'rb'), 'application/ms-excel')
        )

        # Check if the file was successfully attached
        if response.message == 'SUCCESS':
            print(f"Excel file attached to row {row_id} successfully.")
            return True
        else:
            print(f"Error attaching Excel file to row {row_id}.")
            return False

    except Exception as e:
        print(f"Error: {e}")
        return False


In [ ]:
def compare_dataframes(df1, df2, col_name_1, col_name_2):
    # create a compound key based on the unique worklog id and the time worked to enable filtering to find if there has been an update to the time entry for the worklog 
    df1_copy = df1.copy()
    df2_copy = df2.copy()
    df1_copy['compound_loc'] = df1_copy.apply(lambda x: str(x[f'{col_name_1}'])+"_"+str(x[f'{col_name_2}']), axis=1)
    df2_copy['compound_loc'] = df2_copy.apply(lambda x: str(x[f'{col_name_1}'])+"_"+str(x[f'{col_name_2}']), axis=1)
    
    current_record_list = list(set(list(df1_copy.compound_loc)))
    
    updated_records_in_new_df = df2_copy[~df2_copy.compound_loc.isin(current_record_list)]
    
    if len(updated_records_in_new_df) > 0:
        updated_records_in_new_df.drop(columns=['compound_loc'], inplace=True)
    
        return updated_records_in_new_df
    else:
        empty = []
        return empty

In [ ]:
# extract the current records from smartsheet so only new records are added to smartsheet
current_smartsheet_records = extract_and_load_to_dataframe_with_row_id(api_key, monthly_billing)
current_smartsheet_records.head(3)

In [ ]:
# create a compound key using the Research_Info__Account_Num and month_year_of_hours_worked to create a unique ID for each record 
# for the smartsheet data and the Jira data that has been processed so records that have not been added to smartsheet can be identifed
# and so that records that are only in smartsheet can be check to see if there have been changes to the hours_worked_in_month from what 
# has been logged in smartsheet to the newly processsed Jira data

# add the file path for the work log record to each row to a seperate df that will be used for processing the required attachements
file_path_df = result_df.copy()
file_path_df['file_path'] = file_path_df.apply(lambda x: 'delete_me/'+x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked)+'.xlsx', axis=1)
file_path_df['compound'] = file_path_df.apply(lambda x: x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked), axis=1)

current_smartsheet_records = extract_and_load_to_dataframe_with_row_id(api_key, monthly_billing)

# to handle the intial load when there are no current records in smartsheets
if len(current_smartsheet_records) == 0:
    # add records to smartsheet
    add_records_to_smartsheet(result_df, api_key, monthly_billing)
    current_smartsheet_records = extract_and_load_to_dataframe_with_row_id(api_key, monthly_billing)
    
    # create a compound key for linking the smartsheet row id and to identify records that are not currently in smartsheets
    result_df['compound'] = result_df.apply(lambda x: x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked), axis=1)
    current_smartsheet_records['compound'] = current_smartsheet_records.apply(lambda x: x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked), axis=1)
    
    # slice out only the required columns from the smartsheet df
    smartsheet_df_slice = current_smartsheet_records[['row_id', 'compound']]

    # merge the smartsheet slice with the origianl dataframe so the smartsheet row ID is match with each record
    merge_filtered = pd.merge(result_df,smartsheet_df_slice, on='compound', how='left')
    merge_filtered['file_path'] = merge_filtered.apply(lambda x: 'delete_me/'+x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked)+'.xlsx', axis=1)

    # funtion to ensure only int value is available for the row_id
    def row_id_int(data):
        try:
            output = int(data)
        except:
            output = data

        return output

    merge_filtered['row_id'] = merge_filtered.apply(lambda x: row_id_int(x.row_id), axis=1)
    merge_filtered_not_nan = merge_filtered[~merge_filtered.row_id.isna()]
    
    counter = 0
    df_len = len(merge_filtered_not_nan)
    error_list = []

    while counter < df_len:
        try:
            add_excel_file_to_row(api_key, monthly_billing, int(merge_filtered_not_nan.iloc[counter].row_id), merge_filtered_not_nan.iloc[counter].file_path)
            counter = counter+1
        except:
            error_list.append(counter)
            counter = counter+1
    
    
else:
    # create a compound key for linking the smartsheet row id and to identify records that are not currently in smartsheets
    result_df['compound'] = result_df.apply(lambda x: x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked), axis=1)
    current_smartsheet_records['compound'] = current_smartsheet_records.apply(lambda x: x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked), axis=1)

    # get a list of the compound key from smartsheet and use as a filter to identify new Jira records and Jira records that are currently in smartsheet 
    records_in_smartsheet_list = list(set(list(current_smartsheet_records.compound)))
    new_records_to_smartsheets = result_df[~result_df.compound.isin(records_in_smartsheet_list)]
    jira_records_match_smartsheets = result_df[result_df.compound.isin(records_in_smartsheet_list)]
    
    # QC to see if the smartsheet records have time that is different than what the new Jira data batch contains
    qc_monthly_billing = compare_dataframes(current_smartsheet_records, jira_records_match_smartsheets, 'compound', 'hours_worked_in_month')
    if len(qc_monthly_billing) > 0:
        # add records to Variance Report
        qc_monthly_billing.drop(columns=['compound'], inplace=True)
        add_records_to_smartsheet(qc_monthly_billing, api_key, variance_report)
        # add the file attachments
        qc_monthly_billing['compound'] = qc_monthly_billing.apply(lambda x: x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked), axis=1)

        current_smartsheet_records = extract_and_load_to_dataframe_with_row_id(api_key, variance_report)
        current_smartsheet_records['compound'] = current_smartsheet_records.apply(lambda x: x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked), axis=1)
        
        # slice out only the required columns from the smartsheet df
        smartsheet_df_slice = current_smartsheet_records[['row_id', 'compound']]

        # merge the smartsheet slice with the origianl dataframe so the smartsheet row ID is match with each record
        merge_filtered = pd.merge(qc_monthly_billing, smartsheet_df_slice, on='compound', how='left')
        merge_filtered['file_path'] = merge_filtered.apply(lambda x: 'delete_me/'+x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked)+'.xlsx', axis=1)
        
        # funtion to ensure only int value is available for the row_id
        def row_id_int(data):
            try:
                output = int(data)
            except:
                output = data

            return output

        merge_filtered['row_id'] = merge_filtered.apply(lambda x: row_id_int(x.row_id), axis=1)
        merge_filtered_not_nan = merge_filtered[~merge_filtered.row_id.isna()]
        merge_filtered_not_nan.head()

        counter = 0
        df_len = len(merge_filtered_not_nan)
        error_list = []

        while counter < df_len:
            try:
                add_excel_file_to_row(api_key, variance_report, int(merge_filtered_not_nan.iloc[counter].row_id), merge_filtered_not_nan.iloc[counter].file_path)
                counter = counter+1
            except:
                error_list.append(counter)
                counter = counter+1
        
        
        print('*********************************')
        print('*********************************')
        print('*********************************')
        print('')
        print('Records added to Variance Report')
        print('')
        print('*********************************')
        print('*********************************')
        print('*********************************')
        
    # no variance report needed for the monthly billing    
    else:
        print('no variance report needed for the monthly billing')
        
    
    # add records to Monthly Billing Report if available 
    if len(new_records_to_smartsheets) > 0:
        new_records_to_smartsheets.drop(columns=['compound'], inplace=True)
        add_records_to_smartsheet(new_records_to_smartsheets, api_key, monthly_billing)

        new_records_to_smartsheets['compound'] = new_records_to_smartsheets.apply(lambda x: x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked), axis=1)

        current_smartsheet_records = extract_and_load_to_dataframe_with_row_id(api_key, monthly_billing)
        current_smartsheet_records['compound'] = current_smartsheet_records.apply(lambda x: x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked), axis=1)

        # slice out only the required columns from the smartsheet df
        smartsheet_df_slice = current_smartsheet_records[['row_id', 'compound']]

        # merge the smartsheet slice with the origianl dataframe so the smartsheet row ID is match with each record
        merge_filtered = pd.merge(new_records_to_smartsheets, smartsheet_df_slice, on='compound', how='left')
        merge_filtered['file_path'] = merge_filtered.apply(lambda x: 'delete_me/'+x.Research_Info__Account_Num+"_"+str(x.month_year_of_hours_worked)+'.xlsx', axis=1)

        # funtion to ensure only int value is available for the row_id
        def row_id_int(data):
            try:
                output = int(data)
            except:
                output = data

            return output

        merge_filtered['row_id'] = merge_filtered.apply(lambda x: row_id_int(x.row_id), axis=1)
        merge_filtered_not_nan = merge_filtered[~merge_filtered.row_id.isna()]
        merge_filtered_not_nan.head()

        counter = 0
        df_len = len(merge_filtered_not_nan)
        error_list = []

        while counter < df_len:
            try:
                add_excel_file_to_row(api_key, monthly_billing, int(merge_filtered_not_nan.iloc[counter].row_id), merge_filtered_not_nan.iloc[counter].file_path)
                counter = counter+1
            except:
                error_list.append(counter)
                counter = counter+1
    else:
        print('no new records to add to the monthly billing report')

    
    
         
### this section of code loads the records used to aggrogate the monthly billing into a smartsheet table for reference        
        
# check the source_billable_records smartsheet data to see if there are new Jira records that need added 
current_source_billable_smartsheet_records = extract_and_load_to_dataframe_with_row_id(api_key, source_billable_records_sheet)

# if statement to handel inital load
if len(current_source_billable_smartsheet_records) == 0:
    final_record_set['time_record_updated'] = 0
    add_records_to_smartsheet(final_record_set, api_key, source_billable_records_sheet)
    
else:
    current_source_billable_smartsheet_records['WorkLog_ID'] = current_source_billable_smartsheet_records['WorkLog_ID'].astype(int)
    current_source_billable_smartsheet_records['Time_Worked_Hours'] = current_source_billable_smartsheet_records['Time_Worked_Hours'].astype(float)
    billable_worklog_ids_list = list(set(list(current_source_billable_smartsheet_records.WorkLog_ID)))
    
    new_billable_records = final_record_set.copy()
    new_billable_records_to_process = new_billable_records[~new_billable_records.WorkLog_ID.isin(billable_worklog_ids_list)] # these will be loaded into smartsheets
    prior_processes_records = new_billable_records[new_billable_records.WorkLog_ID.isin(billable_worklog_ids_list)] # these will be checked if updates have been made to the time records
    
    if len(new_billable_records_to_process) > 0: 
        print('Load new billable time records to smartsheet')
        new_billable_records_to_process['time_record_updated'] = 0
        add_records_to_smartsheet(new_billable_records_to_process, api_key, source_billable_records_sheet)
    else:
        print('no billable records to add to the report')
    
    # QC to see if time records have been updated and if they have then add them to the table with a flag time_record_updated set to 1
    updated_rows = compare_dataframes(current_source_billable_smartsheet_records, prior_processes_records, 'WorkLog_ID', 'Time_Worked_Hours')
    if len(updated_rows) > 0:
        updated_rows['time_record_updated'] = 1
        # add to smartsheets
        print('Time records have been changed and the record has been posted')
        add_records_to_smartsheet(updated_rows, api_key, source_billable_records_sheet)
    else:
        print('no updates to time records found')
        
        
print('*********************************')
print('*********************************')
print('*********************************')
print('')
print('Reminder to delete the delete me folder')
print('')
print('*********************************')
print('*********************************')
print('*********************************')